In [215]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from IPython.display import display

import gc

In [41]:
games = pd.read_csv('../data/games.csv')
games_details = pd.read_csv('../data/games_details.csv')
players = pd.read_csv('../data/players.csv')
teams = pd.read_csv('../data/teams.csv')
ranking = pd.read_csv('../data/ranking.csv')

In [ ]:
gc.collect()

In [ ]:
fig, ax = plt.subplots(figsize=(11,5))

v_c = games['SEASON'].value_counts().sort_index()
v_c.index = v_c.index.astype(str)
ax.bar(v_c.index, v_c.values)

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(6,5))

v_c = games['HOME_TEAM_WINS'].value_counts(normalize=True)
v_c.index = v_c.index.astype(str)

ax.bar(v_c.index, v_c.values)

for i in ax.patches:
    x, y = i.get_x(), i.get_y()
    w, h = i.get_width(), i.get_height()

    val = str(round(h*100,1)) + '%' 

    ax.text(x + w/2, y + h/2, val,
            horizontalalignment='center', verticalalignment='center',
            fontsize=13, color='white')

plt.show()

In [70]:
def format_record(record):
    w = int(record[0])
    l = int(record[1])
    n = w+l
    
    if n == 0:
        return np.NaN
    
    return w / n

def format_rankings(ranking):
    
    home_record = ranking.loc[:,'HOME_RECORD'].str.split('-').apply(format_record)
    road_record = ranking.loc[:,'ROAD_RECORD'].str.split('-').apply(format_record)
    
    ranking.loc[:,'HOME_RECORD'] = home_record
    ranking.loc[:,'ROAD_RECORD'] = road_record
    
    ranking.loc[:,'SEASON_ID'] = ranking.loc[:,'SEASON_ID'].astype(str).str[1:]
    
    return ranking



ranking = pd.read_csv('../data/ranking.csv')
ranking = format_rankings(ranking)

In [ ]:
games.sample(5)[['GAME_DATE_EST','TEAM_ID_home','TEAM_ID_away','PTS_home','PTS_away','HOME_TEAM_WINS']]

In [146]:
ranking = ranking.sort_values(by='STANDINGSDATE')

In [298]:
def get_team_ranking_before_date(team_id, date, min_games=10):
    
    
    _ranking = ranking.loc[ranking['STANDINGSDATE'] < date]
    _ranking = _ranking.loc[_ranking['TEAM_ID'] == team_id]
    
    if _ranking.tail(1)['G'].values[0] < min_games:
        _ranking = _ranking.loc[_ranking['SEASON_ID']  < _ranking['SEASON_ID'].max()]
    
    _prev_season = _ranking.loc[_ranking['SEASON_ID']  < _ranking['SEASON_ID'].max()]
    _prev_season = _prev_season.loc[_prev_season['STANDINGSDATE'] == _prev_season['STANDINGSDATE'].max()]
    
    _current_season = _ranking[_ranking['STANDINGSDATE'] == _ranking['STANDINGSDATE'].max()]
    
    _current_season = _current_season[['TEAM_ID','G','W_PCT','HOME_RECORD','ROAD_RECORD']]
    _prev_season = _prev_season[['TEAM_ID','W_PCT','HOME_RECORD','ROAD_RECORD']]
    
    return _current_season.merge(_prev_season, on='TEAM_ID', suffixes=('','_prev')).drop(columns='TEAM_ID')


def get_team_ranking_before_game(games):
    _games = games.copy()
    
    def _get_ranking(game):
        date = game['GAME_DATE_EST'].values[0]
        home_team = game['TEAM_ID_home'].values[0]
        away_team = game['TEAM_ID_away'].values[0]
        
        h_rank = get_team_ranking_before_date(home_team, date)
        a_rank = get_team_ranking_before_date(away_team, date)
        
        h_rank.columns += '_home'
        a_rank.columns += '_away'
        
        return pd.concat([h_rank, a_rank], axis=1)
    
        
    _games = _games.groupby('GAME_ID').apply(_get_ranking)
    _games = _games.reset_index().drop(columns='level_1')
    
    return _games.reset_index(drop=True)

In [ ]:
%%time

_games = games.sample(10)
get_team_ranking_before_game(_games)

In [300]:
games = games.sort_values(by='GAME_DATE_EST')

In [336]:
def get_games_stats_before_date(team_id, date, n, stats_cols, game_type='all'):
    """
    """
    
    if game_type not in ['all','home','away']:
        raise ValueError('game_type must be all, home or away')
    
    _games = games.loc[games['GAME_DATE_EST'] < date]
    _games = _games.loc[(_games['TEAM_ID_home'] == team_id) | (_games['TEAM_ID_away'] == team_id)]
    
    _games.loc[:,'is_home'] = _games['TEAM_ID_home'] == team_id
    
    if game_type == 'home':
        _games = _games.loc[_games['is_home']]
        
    elif game_type == 'away':
        _games = _games.loc[~_games['is_home']]
        
    _games.loc[:,'WIN_PRCT'] = _games['is_home'] == _games['HOME_TEAM_WINS']
    
    for col in stats_cols:
        _games.loc[:,col] = np.where(_games['is_home'], _games['%s_home'%col], _games['%s_away'%col])
    
    cols = ['WIN_PRCT'] + stats_cols
    
    if len(_games) < n:
        return _games[cols]
    
    return _games.tail(n)[cols]



STATS_COLUMNS = ['PTS','FG_PCT','FT_PCT','FG3_PCT','AST','REB']


def get_games_stats_before_game(games, n, stats_cols=STATS_COLUMNS):
    _games = games.copy()
    
    def _get_stats(game):
        date = game['GAME_DATE_EST'].values[0]
        home_team = game['TEAM_ID_home'].values[0]
        away_team = game['TEAM_ID_away'].values[0]
        
        h_stats = get_games_stats_before_date(home_team, date, n, stats_cols, game_type='all')
        h_stats.columns += '_home_%ig'%n
        h_stats = h_stats.mean().to_frame().T
        
        a_stats = get_games_stats_before_date(away_team, date, n, stats_cols, game_type='all')
        a_stats.columns += '_away_%ig'%n
        a_stats = a_stats.mean().to_frame().T
        
        return pd.concat([h_stats, a_stats], axis=1)
        
        
    _games = _games.groupby('GAME_ID').apply(_get_stats)
    _games = _games.reset_index().drop(columns='level_1')
    
    return _games.reset_index(drop=True)

In [ ]:
%%time

_games = games.sample(10, random_state=42)
get_games_stats_before_game(_games, n=2)

In [ ]:
one_game = games.sample(1, random_state=42)
one_game[['GAME_ID','GAME_DATE_EST','TEAM_ID_home','TEAM_ID_away','PTS_home','PTS_away','HOME_TEAM_WINS']]

In [325]:
date = '2013-04-01'
game_id = 21201104
h_team = 1610612745
a_team = 1610612753

In [ ]:
_games = games[games['GAME_DATE_EST'] < date].sort_values(by='GAME_DATE_EST')
    
for team_id in [h_team, a_team]:
    print(team_id)
    _games2 = _games.loc[(_games['TEAM_ID_home'] == team_id) | (_games['TEAM_ID_away'] == team_id)]

    
    _games2 = _games2[['GAME_DATE_EST','TEAM_ID_home','PTS_home','REB_home','TEAM_ID_away','PTS_away','REB_away']]

 
    display(_games2.tail(5))

In [ ]:
res = get_games_stats_before_game(one_game, n=5, stats_cols=['PTS','REB'])
res

In [ ]:
h_pts = 116 + 96 + 91 + 94 + 98
a_pts = 89 + 94 + 108 + 97 + 88

h_reb = 54 + 44 + 42 + 35 + 48
a_reb = 44 + 43 + 34 + 41 + 53

h_pts = h_pts / 5
h_reb = h_reb / 5

a_pts = a_pts / 5
a_reb = a_reb / 5

print(h_pts, h_reb, a_pts, a_reb)

assert h_pts == res['PTS_home_5g'].values[0]
assert h_reb == res['REB_home_5g'].values[0]
assert a_pts == res['PTS_away_5g'].values[0]
assert a_reb == res['REB_away_5g'].values[0]


In [ ]:
%%time 

def prepare_games_data(games):
    rank_stats = get_team_ranking_before_game(games)
    
    game_stats_3g = get_games_stats_before_game(games, n=3)
    
    game_stats_10g = get_games_stats_before_game(games, n=10)
    
    formated_games = rank_stats.merge(game_stats_3g, on='GAME_ID')
    formated_games = formated_games.merge(game_stats_10g, on='GAME_ID')
    
    return formated_games
    
    
_games = games.sample(10, random_state=42)
prepare_games_data(_games)

In [363]:
_games = games[games['SEASON'] >= 2007]

In [ ]:
_games['SEASON'].unique()

In [ ]:
%%time
games_formated = prepare_games_data(_games)

In [385]:
games_formated = games_formated.merge(games[['GAME_ID','GAME_DATE_EST','SEASON','HOME_TEAM_WINS']], on='GAME_ID', how='left')
games_formated = games_formated.reset_index(drop=True)

In [ ]:
games_formated.head()

In [ ]:
games_formated.info()

In [ ]:
games_formated.isna().sum().sum()

In [388]:
games_formated.to_csv('games_formated_07_20.csv', index=False)